<a href="https://colab.research.google.com/github/Agwen12/imperative_C/blob/master/Kopia_notatnika_DeepVsWide_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.7.11


In [ ]:
# TEst colaba

# Szymon
# Mikołaj
# Paweł
# Paulina
# Dawid

In [ ]:
!pip install torch torchvision 

In [ ]:
from __future__ import print_function
import argparse
from math import ceil
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchsummary import summary
from torchvision import datasets, transforms
from torch.autograd import Variable

Dataset

In [ ]:
BATCH_SIZE = 1000

class Flatten(object):
    """ Transforms a PIL image to a flat numpy array. """
    def __call__(self, sample):
        return np.array(sample, dtype=np.float32).flatten()

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=Flatten()),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=Flatten()),
    batch_size=BATCH_SIZE, shuffle=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Model definition

In [ ]:
class Metrics():
  def __init__(self):
    self.saturation = None
    self.entropy = None
    self.lmc = None
    self.winning_tickets = None

In [ ]:
# Experiment

class Experiment(torch.nn.Module):
  def __init__(self, x_dim, y_dim, wide_layer_dim, deep_layer_dim, layers_count):
    super(Experiment, self).__init__()
    self.layers = [torch.nn.Linear(x_dim, wide_layer_dim)]
    self.linear_1 = torch.nn.Linear(x_dim, 64)
    if layers_count > 1:
      self.layers.append(torch.nn.Linear(wide_layer_dim, deep_layer_dim))

      for layer in range(layers_count - 2):
        self.layers.append(torch.nn.Linear(deep_layer_dim, deep_layer_dim))

      self.layers.append(torch.nn.Linear(deep_layer_dim, y_dim))
    else:
      self.layers.append(torch.nn.Linear(wide_layer_dim, y_dim))

    # It is required by pytorch to store layers in submodule
    self.linears = nn.ModuleList(self.layers)

  def forward(self, x):
    h = F.relu(self.layers[0](x))

    for layer in self.layers[1:-1]:
      h = F.relu(layer(h))

    return F.softmax(self.layers[-1](h), dim=1)

  def metric(self, epoch):
    pass

  def plot_saturation(self):
    for p in self.layers:
      plt.figure()
      plt.hist(p.weight.detach().numpy().reshape((-1,)), bins=50)
      plt.show()
  

In [ ]:
def prepareExperiment(number_of_parameters, net_depth, net_input_size=784, net_output_size=10):
  wide_layer_dim = ceil(number_of_parameters / net_input_size)
  deep_layer_dim = 0

  if net_depth == 1:
    return Experiment(net_input_size, net_output_size, wide_layer_dim, deep_layer_dim, net_depth)

  # In first step determine correct size of wide layer first
  for neurons_count in range(1,wide_layer_dim):
    if net_input_size * neurons_count + neurons_count ** 2 + neurons_count * net_output_size > number_of_parameters:
      wide_layer_dim = neurons_count + 1
      deep_layer_dim = neurons_count + 1
      break
    
  # Once number of neurons from first layer is determined, just align deeper layers
  if net_depth > 2:
    for neurons_count in range(1,deep_layer_dim):
      number_of_input_parameters = net_input_size * wide_layer_dim 
      number_of_output_parameters =  neurons_count * net_output_size
      number_of_middle_parameters = wide_layer_dim * neurons_count + neurons_count ** 2 * (net_depth - 1)
      if number_of_input_parameters + number_of_middle_parameters + number_of_output_parameters > number_of_parameters:
        deep_layer_dim = neurons_count + 1
        break

  return Experiment(net_input_size, net_output_size, wide_layer_dim, deep_layer_dim, net_depth)

In [ ]:
model_wide = prepareExperiment(50000, 1)
summary(model_wide, (1, 784))

model_deep = prepareExperiment(50000, 6)
summary(model_deep, (1, 784))

model = model_deep


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]          50,240
            Linear-2                [-1, 1, 10]             650
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.19
Estimated Total Size (MB): 0.20
----------------------------------------------------------------
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 60]          47,100
            Linear-2                [-1, 1, 20]           1,220
            Linear-3                [-1, 1, 20]             420
            Linear-4                [-1, 1, 20]             420
            Linear-5                [-1, 1,

Model training

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# training hyperparameters
IN_DIM = 28 * 28
#IN_DIM = 3 * 10
OUT_DIM = 10
LR = 10 ** -4
BATCH_SIZE = 32
EPOCHS = 10
# contour plot resolution
STEPS = 40

In [ ]:
# Training and Metrics gathering (TODO)

optimizer = optim.Adam(model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.099306
Train Epoch: 1 [1000/60000 (2%)]	Loss: -0.102218
Train Epoch: 1 [2000/60000 (3%)]	Loss: -0.101971
Train Epoch: 1 [3000/60000 (5%)]	Loss: -0.102361
Train Epoch: 1 [4000/60000 (7%)]	Loss: -0.104294
Train Epoch: 1 [5000/60000 (8%)]	Loss: -0.105823
Train Epoch: 1 [6000/60000 (10%)]	Loss: -0.104053
Train Epoch: 1 [7000/60000 (12%)]	Loss: -0.106240
Train Epoch: 1 [8000/60000 (13%)]	Loss: -0.104951
Train Epoch: 1 [9000/60000 (15%)]	Loss: -0.109981
Train Epoch: 1 [10000/60000 (17%)]	Loss: -0.111581
Train Epoch: 1 [11000/60000 (18%)]	Loss: -0.112057
Train Epoch: 1 [12000/60000 (20%)]	Loss: -0.111735
Train Epoch: 1 [13000/60000 (22%)]	Loss: -0.112415
Train Epoch: 1 [14000/60000 (23%)]	Loss: -0.114103
Train Epoch: 1 [15000/60000 (25%)]	Loss: -0.113858
Train Epoch: 1 [16000/60000 (27%)]	Loss: -0.116536
Train Epoch: 1 [17000/60000 (28%)]	Loss: -0.116676
Train Epoch: 1 [18000/60000 (30%)]	Loss: -0.118146
Train Epoch: 1 [19000/60000 (32%)]	Loss: -0.118962

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: -0.2256, Accuracy: 2262/10000 (23%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -0.235010
Train Epoch: 2 [1000/60000 (2%)]	Loss: -0.230584
Train Epoch: 2 [2000/60000 (3%)]	Loss: -0.228462
Train Epoch: 2 [3000/60000 (5%)]	Loss: -0.216057
Train Epoch: 2 [4000/60000 (7%)]	Loss: -0.228555
Train Epoch: 2 [5000/60000 (8%)]	Loss: -0.239776
Train Epoch: 2 [6000/60000 (10%)]	Loss: -0.243651
Train Epoch: 2 [7000/60000 (12%)]	Loss: -0.239473
Train Epoch: 2 [8000/60000 (13%)]	Loss: -0.245324
Train Epoch: 2 [9000/60000 (15%)]	Loss: -0.250552
Train Epoch: 2 [10000/60000 (17%)]	Loss: -0.268481
Train Epoch: 2 [11000/60000 (18%)]	Loss: -0.248280
Train Epoch: 2 [12000/60000 (20%)]	Loss: -0.250722
Train Epoch: 2 [13000/60000 (22%)]	Loss: -0.271529
Train Epoch: 2 [14000/60000 (23%)]	Loss: -0.263973
Train Epoch: 2 [15000/60000 (25%)]	Loss: -0.265095
Train Epoch: 2 [16000/60000 (27%)]	Loss: -0.269642
Train Epoch: 2 [17000/60000 (28%)]	Loss: -0.276766
Train Epoch: 2 [18000/60000 (30%)]	Loss

Results

In [ ]:
# First idea test, accuracy after 10th epoch
# Deep: Test set: Average loss: -0.7717, Accuracy: 7757/10000 (78%)
# Wide: Test set: Average loss: -0.9187, Accuracy: 9201/10000 (92%)

# TODO other metrics